In [3]:
import os

HOME = os.getcwd()
print(HOME)

%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino
/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 421, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 421 (delta 144), reused 126 (delta 126), pack-reused 232
Receiving objects: 100% (421/421), 12.85 MiB | 5.93 MiB/s, done.
Resolving deltas: 100% (216/216), done.
/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino/GroundingDINO
/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [5]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha2/groundingdino_swinb_cogcoor.pth

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino
mkdir: /Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino/weights: File exists
/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino/weights
/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino/weights/groundingdino_swint_ogc.pth ; exist: False


In [8]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate
from groundingdino.util.inference import Model

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

GROUNDING_DINO_CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH))

GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", "groundingdino_swinb_cogcoor.pth")
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))

grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino/GroundingDINO
final text_encoder_type: bert-base-uncased


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jc/code/aipi540-module1/notebooks/dino/GroundingDINO/groundingdino/weights/groundingdino_swint_ogc.pth'

In [ ]:
import torch
from torchvision.ops import box_convert
import numpy as np
from PIL import Image
import supervision as sv

In [ ]:
IMAGE_PATH = "_computer_vision/object_detection/data/images/traditionalgdino.png"
TEXT_PROMPT = "lamp, rug, bed, sofa, couch, dresser, chair, bench, table, vase"
BOX_TRESHOLD = 0.25
TEXT_TRESHOLD = 0.25

image_source, image = load_image(IMAGE_PATH)

boxes, logits, phrases = predict(
    model=model,
    image=image,
    caption=TEXT_PROMPT,
    box_threshold=BOX_TRESHOLD,
    text_threshold=TEXT_TRESHOLD
)

annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)

%matplotlib inline
sv.plot_image(annotated_frame, (16, 16))